# Save sample data for testing

In [1]:
import boto3
import os
from tqdm import tqdm

s3 = boto3.client("s3")

# List all objects helper internal function
def list_all_objects(bucket, prefix):
    # Create a paginator for list_objects_v2
    paginator = s3.get_paginator('list_objects_v2')

    # Use the paginator to iterate through all pages
    all_objects = []
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        if 'Contents' in page:
            all_objects.extend(page['Contents'])

    return all_objects

bucket = 'udacity-deeplearning-project'
prefix = 'data'
sample_prefix = 'sample'

all_files = list_all_objects(bucket, prefix)

sample_files = [file_meta for file_meta in all_files if '001.Affenpinscher' in file_meta.get("Key")]

for file_meta in tqdm(sample_files):
    key = file_meta.get("Key")
    
    # Move the data from s3 to a different prefix
    dirname = os.path.dirname(key)
    if dirname:
        os.makedirs(dirname, exist_ok=True)

    s3.download_file(bucket, key, key)
    # s3.copy_object(
    #     Bucket=bucket,
    #     CopySource=f"{bucket}/{key}",
    #     Key=f"{sample_prefix}/{key}"
    # )

100%|██████████| 80/80 [00:07<00:00, 10.92it/s]


## Testing Estimator Locally Prior to Deployment to ECR

In [5]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
import sagemaker
import subprocess

subprocess.run(['sh', 'docker-build.sh'])

role=get_execution_role()

bucket = 'udacity-deeplearning-project'
sample_prefix = 'sample'

hyperparameters = {
    'model-type': 'vgg16',
    'num-classes': 133,
    'batch-size': 32,
    'lr': 0.005070970373087015
}

bucket = 'udacity-deeplearning-project'
s3_output_location = f"s3://{bucket}/outputs"

estimator=Estimator(
    image_uri='udacity-sagemaker-hpo',
    role=role,
    instance_count=1,
    instance_type='local',
    output_path=s3_output_location,
    hyperparameters=hyperparameters
)

model_inputs = {
    "train": "file://./data/test",
    "test": "file://./data/valid"
}

estimator.fit(inputs=model_inputs, logs=True ,wait=True)

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sha256:de7a6ff5d9bb2950d37a327228b33e8e1ee45339b81518da342c9f32c44424e9
REPOSITORY              TAG       IMAGE ID       CREATED                  SIZE
udacity-sagemaker-hpo   latest    de7a6ff5d9bb   Less than a second ago   3.92GB
<none>                  <none>    4a21c8e53d63   4 minutes ago            3.92GB
<none>                  <none>    0814594320ea   8 minutes ago            3.92GB


INFO:sagemaker:Creating training-job with name: udacity-sagemaker-hpo-2024-11-26-13-16-44-757
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' is not installed. Proceeding to check for 'docker-compose' CLI.
INFO:sagemaker.local.image:'Docker Compose' found using Docker Compose CLI.
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.i

 Container t4grlud1y8-algo-1-b9y75  Creating
 Container t4grlud1y8-algo-1-b9y75  Created
Attaching to t4grlud1y8-algo-1-b9y75
t4grlud1y8-algo-1-b9y75  | sed: can't read changehostname.c: No such file or directory
t4grlud1y8-algo-1-b9y75  | gcc: error: changehostname.c: No such file or directory
t4grlud1y8-algo-1-b9y75  | gcc: fatal error: no input files
t4grlud1y8-algo-1-b9y75  | compilation terminated.
t4grlud1y8-algo-1-b9y75  | gcc: error: changehostname.o: No such file or directory
t4grlud1y8-algo-1-b9y75  | ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
t4grlud1y8-algo-1-b9y75  | 2024-11-26 13:16:45,821 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
t4grlud1y8-algo-1-b9y75  | 2024-11-26 13:16:45,824 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
t4grlud1y8-algo-1-b9y75  | 2024-11-26 13:16:45,825 sagemaker-training-toolkit INFO

INFO:sagemaker.local.image:===== Job Complete =====


## Testing Deployed Estimator

In [10]:
!sh push-container.sh

push-container.sh: line 1: fg: no job control
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (1/1) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 2B                                         0.0s
ERROR: failed to solve: failed to read dockerfile: open Dockerfile: no such file or directory
The push refers to repository [598308907998.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo]

bf18a086: Preparing 
77796a3a: Preparing 
b282a0ff: Preparing 
95045e04: Preparing 
e9708ca1: Preparing 
8f6060c6: Preparing 
a3c12226: Preparing 
62daa95e: Preparing 
8fe1cb59: Preparing 
061a5b0d:

In [7]:
model_inputs = {
    "train": sagemaker.inputs.TrainingInput(
        s3_data=f"s3://{bucket}/{sample_prefix}/data/train/",
        content_type="application/x-image"
    ),
    "test": sagemaker.inputs.TrainingInput(
        s3_data=f"s3://{bucket}/{sample_prefix}/data/valid/",
        content_type="application/x-image"
    )
}

In [8]:
hyperparameters = {
    'model-type': 'vgg16',
    'num-classes': 133,
    'batch-size': 32,
    'lr': 0.005070970373087015
}

In [9]:
estimator=Estimator(
    image_uri='598308907998.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    hyperparameters=hyperparameters
)

estimator.fit(inputs=model_inputs, logs=True)

INFO:sagemaker:Creating training-job with name: udacity-sagemaker-hpo-2024-11-26-13-19-14-190


2024-11-26 13:19:16 Starting - Starting the training job...
2024-11-26 13:19:30 Starting - Preparing the instances for training...
2024-11-26 13:20:01 Downloading - Downloading input data...
2024-11-26 13:20:21 Downloading - Downloading the training image...
2024-11-26 13:21:02 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
2024-11-26 13:21:11,464 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-11-26 13:21:11,466 sagemaker-training-toolkit INFO     No GPUs detec

In [ ]:
sagemaker_client = boto3.client("sagemaker")

# Get the latest training job name
training_job_name = estimator.latest_training_job.name
print(f"Training Job Name: {training_job_name}")

# Get the model artifact location from the training job details
response = sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
model_artifact = response["ModelArtifacts"]["S3ModelArtifacts"]
print(f"Model Artifact Location: {model_artifact}")


# Testing Batch Inference

In [28]:
from sagemaker.pytorch import PyTorchModel

pytorch_model = PyTorchModel(
    model_data=model_artifact,  # Use the artifact location from the training job
    role=estimator.role,       # Use the role from your estimator
    entry_point="image/inference.py",
    framework_version="1.8.1",
    py_version="py3",
    image_uri="763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-cpu-py36-ubuntu18.04",  # PyTorch-Inference image
)


In [29]:
transformer = pytorch_model.transformer(
    instance_count=1,
    instance_type="ml.m5.large",
    output_path="s3://udacity-deeplearning-project/inference/",  # S3 location for output
)


INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-598308907998/udacity-sagemaker-hpo-2024-11-24-20-20-00-150/output/model.tar.gz), script artifact (None), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-598308907998/pytorch-inference-2024-11-24-20-26-25-124/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2024-11-24-20-26-29-479


In [30]:
transformer.transform(
    data="s3://udacity-deeplearning-project/sample/data/batch/",
    content_type="application/x-image",
    wait=True
)

print("Batch transform completed. Check the output in the specified S3 bucket.")

INFO:sagemaker:Creating transform job with name: pytorch-inference-2024-11-24-20-26-32-239


..............................2024-11-24 20:31:35,030 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2024-11-24 20:31:35,237 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.4.0
TS Home: /opt/conda/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 2
Max heap size: 952 M
Python executable: /opt/conda/bin/python3.6
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Metrics address: http://127.0.0.1:8082
Model Store: /.sagemaker/ts/models
Initial Models: model.mar
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 2
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Prefer direct buffer: false
Allowed Urls: [file://.*|http(s)?://.*]
Custom python dependency for model allowed: false
Metrics report forma

# END OF NOTEBOOK